# Imports

In [25]:
import cv2
import numpy as np
import matplotlib.pyplot as plt
import pytesseract
import re

pytesseract.pytesseract.tesseract_cmd = "C:/Program Files/Tesseract-OCR/tesseract.exe"

# Opening the image

In [26]:
image = cv2.imread("./img/list_3.jpg")

def blur_and_threshold(gray):
    gray = cv2.GaussianBlur(gray, (3, 3), 2)
    threshold = cv2.adaptiveThreshold(gray, 255, cv2.ADAPTIVE_THRESH_GAUSSIAN_C, cv2.THRESH_BINARY, 11, 2)
    threshold = cv2.fastNlMeansDenoising(threshold, 11, 31, 9)
    return threshold


# ## **Find the Biggest Contour**

# **Note: We made sure the minimum contour is bigger than 1/10 size of the whole picture. This helps in removing very small contours (noise) from our dataset**


def biggest_contour(contours, min_area):
    biggest = None
    max_area = 0
    biggest_n = 0
    approx_contour = None
    for n, i in enumerate(contours):
        area = cv2.contourArea(i)

        if area > min_area / 10:
            peri = cv2.arcLength(i, True)
            approx = cv2.approxPolyDP(i, 0.02 * peri, True)
            if area > max_area and len(approx) == 4:
                biggest = approx
                max_area = area
                biggest_n = n
                approx_contour = approx

    return biggest_n, approx_contour


def order_points(pts):
    # initialzie a list of coordinates that will be ordered
    # such that the first entry in the list is the top-left,
    # the second entry is the top-right, the third is the
    # bottom-right, and the fourth is the bottom-left
    pts = pts.reshape(4, 2)
    rect = np.zeros((4, 2), dtype="float32")

    # the top-left point will have the smallest sum, whereas
    # the bottom-right point will have the largest sum
    s = pts.sum(axis=1)
    rect[0] = pts[np.argmin(s)]
    rect[2] = pts[np.argmax(s)]

    # now, compute the difference between the points, the
    # top-right point will have the smallest difference,
    # whereas the bottom-left will have the largest difference
    diff = np.diff(pts, axis=1)
    rect[1] = pts[np.argmin(diff)]
    rect[3] = pts[np.argmax(diff)]

    # return the ordered coordinates
    return rect


# ## Find the exact (x,y) coordinates of the biggest contour and crop it out


def four_point_transform(image, pts):
    # obtain a consistent order of the points and unpack them
    # individually
    rect = order_points(pts)
    (tl, tr, br, bl) = rect

    # compute the width of the new image, which will be the
    # maximum distance between bottom-right and bottom-left
    # x-coordiates or the top-right and top-left x-coordinates
    widthA = np.sqrt(((br[0] - bl[0]) ** 2) + ((br[1] - bl[1]) ** 2))
    widthB = np.sqrt(((tr[0] - tl[0]) ** 2) + ((tr[1] - tl[1]) ** 2))
    maxWidth = max(int(widthA), int(widthB))

    # compute the height of the new image, which will be the
    # maximum distance between the top-right and bottom-right
    # y-coordinates or the top-left and bottom-left y-coordinates
    heightA = np.sqrt(((tr[0] - br[0]) ** 2) + ((tr[1] - br[1]) ** 2))
    heightB = np.sqrt(((tl[0] - bl[0]) ** 2) + ((tl[1] - bl[1]) ** 2))
    maxHeight = max(int(heightA), int(heightB))

    # now that we have the dimensions of the new image, construct
    # the set of destination points to obtain a "birds eye view",
    # (i.e. top-down view) of the image, again specifying points
    # in the top-left, top-right, bottom-right, and bottom-left
    # order
    dst = np.array([
        [0, 0],
        [maxWidth - 1, 0],
        [maxWidth - 1, maxHeight - 1],
        [0, maxHeight - 1]], dtype="float32")

    # compute the perspective transform matrix and then apply it
    M = cv2.getPerspectiveTransform(rect, dst)
    warped = cv2.warpPerspective(image, M, (maxWidth, maxHeight))

    # return the warped image
    return warped


# # Transformation the image

# **1. Convert the image to grayscale**

# **2. Remove noise and smoothen out the image by applying blurring and thresholding techniques**

# **3. Use Canny Edge Detection to find the edges**

# **4. Find the biggest contour and crop it out**


def transformation(image):
    image = image.copy()

    height, width, channels = image.shape
    gray = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)
    image_size = gray.size

    threshold = blur_and_threshold(gray)
    # We need two threshold values, minVal and maxVal. Any edges with intensity gradient more than maxVal
    # are sure to be edges and those below minVal are sure to be non-edges, so discarded.
    #  Those who lie between these two thresholds are classified edges or non-edges based on their connectivity.
    # If they are connected to "sure-edge" pixels, they are considered to be part of edges.
    #  Otherwise, they are also discarded
    edges = cv2.Canny(threshold, 50, 150, apertureSize=7)
    contours, hierarchy = cv2.findContours(edges, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)
    simplified_contours = []

    for cnt in contours:
        hull = cv2.convexHull(cnt)
        poly = cv2.approxPolyDP(hull, 0.001 * cv2.arcLength(hull, True), True)
        if poly.shape[0] != 4:
            continue
        simplified_contours.append(poly)
    simplified_contours = np.array(simplified_contours)
    biggest_n, approx_contour = biggest_contour(simplified_contours, image_size)
 
    threshold = cv2.drawContours(image, simplified_contours, -1, (0, 0, 0), 1)
    
    dst = 0
    if approx_contour is not None and len(approx_contour) == 4:
        approx_contour = np.float32(approx_contour)
        dst = four_point_transform(threshold, approx_contour)
    croppedImage = dst
    return croppedImage


# **Increase the brightness of the image by playing with the "V" value (from HSV)**

def increase_brightness(img, value=30):
    hsv = cv2.cvtColor(img, cv2.COLOR_BGR2HSV)
    h, s, v = cv2.split(hsv)
    lim = 255 - value
    v[v > lim] = 255
    v[v <= lim] += value
    final_hsv = cv2.merge((h, s, v))
    img = cv2.cvtColor(final_hsv, cv2.COLOR_HSV2BGR)
    return img


# **Sharpen the image using Kernel Sharpening Technique**


def final_image(rotated):
    # Create our shapening kernel, it must equal to one eventually
    kernel_sharpening = np.array([[0, -1, 0],
                                  [-1, 5, -1],
                                  [0, -1, 0]])
    # applying the sharpening kernel to the input image & displaying it.
    sharpened = cv2.filter2D(rotated, -1, kernel_sharpening)
    sharpened = increase_brightness(sharpened, 30)
    return sharpened

blurred_threshold = transformation(image)
image = final_image(blurred_threshold)


# Paper sheet detection

# Reverse perspective transform

# Table detection

In [34]:
denoised = cv2.fastNlMeansDenoising(image, None, 10) # TODO Parameter

gray = cv2.cvtColor(denoised, cv2.COLOR_BGR2GRAY)

# gray_blurred = cv2.GaussianBlur(gray, (0,0), 3)
# gray = cv2.addWeighted(gray, 1.5, gray_blurred, -0.5, 0)

_, binary = cv2.threshold(gray, 128, 255, cv2.THRESH_OTSU)
binary = 255 - binary

vertical_kernel = cv2.getStructuringElement(cv2.MORPH_CROSS,(1, binary.shape[1] // 150))

binary_vertical = cv2.erode(binary, vertical_kernel, iterations=7)
binary_vertical = cv2.dilate(binary_vertical, vertical_kernel, iterations=7)


horizontal_kernel = cv2.getStructuringElement(cv2.MORPH_CROSS,(binary.shape[1] // 150, 1))

binary_horizontal = cv2.erode(binary, horizontal_kernel, iterations=7)
binary_horizontal = cv2.dilate(binary_horizontal, horizontal_kernel, iterations=7)

vh_lines = cv2.addWeighted(binary_vertical, 0.5, binary_horizontal, 0.5, 0)

kernel = cv2.getStructuringElement(cv2.MORPH_CROSS,(3,3))
vh_lines = cv2.erode(~vh_lines, kernel, iterations=3)

_, vh_lines = cv2.threshold(vh_lines, 128, 255, cv2.THRESH_OTSU | cv2.THRESH_BINARY)
# vh_lines = cv2.cvtColor(vh_lines, cv2.COLOR_GRAY2RGB)
# image_vh_lines = cv2.bitwise_not(cv2.bitwise_xor(image, vh_lines))

# Finding contours

contours, hierachy = cv2.findContours(vh_lines, cv2.RETR_TREE, cv2.CHAIN_APPROX_SIMPLE)

bounding_boxes = [cv2.boundingRect(c) for c in contours]
(contours, bounding_boxes) = zip(*sorted(zip(contours, bounding_boxes),key=lambda x:x[1][1]))

boxes: list[list[int]] = []
for contour in contours:
    x, y, w, h = cv2.boundingRect(contour)
    # image_vh_lines = cv2.rectangle(image_vh_lines, (x,y), (x+w,y+h), (0, 255, 0), 2)
    boxes.append([x,y,w,h])


columns = []
rows = []

mean_height = np.mean([box[3] for box in boxes])
columns.append(boxes[0])
previous = boxes[0]
for box in boxes:
    if box[1] <= (previous[1] + mean_height / 2):
        columns.append(box)
        previous = box
        if box is boxes[-1]:
            rows.append(columns)
    else:
        rows.append(columns)
        columns = []
        previous = box
        columns.append(box)
else:
    rows.append(columns)
    
for idx, row in enumerate(rows):
    rows[idx] = list(sorted(row, key=lambda r: r[0]))

# image_2 = image.copy()

names = []
delta = 1

min_x = rows[0][1][0] - 10

for idx, row in enumerate(rows):    
    if len(row) < 2:
        continue
        
    x, y, w, h = row[1]    
    name_column = binary[y+delta:y+h-delta, x+delta:x+w-delta]
    
        
    if x < min_x:
        rows[idx] = row[1:]
        x, y, w, h = rows[idx][1] 
        name_column = binary[y+delta:y+h-delta, x+delta:x+w-delta]
    
    scaling = 2
    new_shape = (int(name_column.shape[1] * scaling), int(name_column.shape[0] * scaling))
    name_column = cv2.resize(name_column, new_shape)
    name_column = cv2.morphologyEx(name_column, cv2.MORPH_CLOSE, cv2.getStructuringElement(cv2.MORPH_CROSS, (3,3)), iterations=1)
    _, name_column = cv2.threshold(name_column, 128, 255, cv2.THRESH_BINARY)

    w, h = name_column.shape
    d = delta * 2
    
    name_column = name_column[d:w-d, d:h-d]

    if name_column.size == 0:
        continue    
        
    """
    scaling = 10
    new_shape = (int(name_column.shape[1] * scaling), int(name_column.shape[0] * scaling))
    name_column = cv2.resize(name_column, new_shape)
    
    name_column = cv2.fastNlMeansDenoising(name_column, None, 10)
    
    name_column = cv2.morphologyEx(name_column, cv2.MORPH_CLOSE, cv2.getStructuringElement(cv2.MORPH_RECT, (3,3)), iterations=1)
    name_blurred = cv2.GaussianBlur(name_column, (0,0), 3)
    name_column = cv2.addWeighted(name_column, -0.5, name_blurred, 1.5, 0)

    _, name_column = cv2.threshold(name_column, 128, 255, cv2.THRESH_OTSU)
    """
    name_column = 255 - name_column
    
    # cv2.imwrite("./img/output.png", name_column)
    
    name = pytesseract.image_to_string(name_column, lang="eng")
    name = name.strip()
    regex = re.compile('[^a-zA-Z ,]')
    name = regex.sub("", name)
    names.append(name)
    
names
    
# cv2.imwrite("./img/output.png", gray)

['Intelligente Systeme TIK',
 '',
 'Baumann, Lysann',
 'Beetz, Robin Georg',
 'Beuerle, Marco',
 'Domitrovic, Max',
 'Druica, Mathias',
 'Egger, Julia',
 'Fischer, David',
 'Fisher, Jamie',
 'Gmeiner, Leander Gabriel Mauritius',
 'Handschuh, Jannik ',
 'Hogan, Finley ',
 'Kiele, Milan',
 'Marschall, Linus',
 'Medwedkin, Eduard',
 'Naas, Jasper',
 'Nusch, Hannes',
 'Rottweiler, Philipp',
 'Schilling, Tobias',
 'Schneider, AnnaSophie',
 'Seidel, Yannick',
 'Siegert, Daniel Valentin',
 'Zagst, Jonas',
 '',
 '']

In [36]:
o_img = binary - (255 - vh_lines)
names_signatures = list(filter(lambda x: x[0] and len(x[1]) > 2, zip(names, rows)))

mean_height = np.mean([row[2][3] for _, row in names_signatures])

kernel = cv2.getStructuringElement(cv2.MORPH_RECT, (18, 7))

o_img = cv2.cvtColor(o_img, cv2.COLOR_GRAY2RGB)
o_img_c= o_img.copy()

results: dict[str, bool] = {}
for name, row in names_signatures[1:]:
    x, y, w, h = row[2]
    # h += int(mean_height) // 3
    # y -= int(mean_height) // 6
    
    # o_img = o_img[y:y+h, x:x+w]
    
    # o_img = cv2.adaptiveThreshold(o_img, 255, cv2.ADAPTIVE_THRESH_GAUSSIAN_C, cv2.THRESH_BINARY, 11, 2)
    # o_img = cv2.fastNlMeansDenoising(o_img, None, 180)
    o_img_sec = cv2.Canny(o_img[y:y+h, x:x+w], 128, 255)
    
    o_img_sec = cv2.morphologyEx(o_img_sec, cv2.MORPH_CLOSE, kernel, iterations=2)
    
    cnts, hier = cv2.findContours(o_img_sec, cv2.RETR_LIST, cv2.CHAIN_APPROX_SIMPLE, offset=(x,y))
    
    filtered_cnts = []
    for contour in cnts:
        cx, cy, cw, ch = cv2.boundingRect(contour)
        
        # o_img_c = cv2.rectangle(o_img_c, (cx,cy), (cx+cw,cy+ch), (0, 255, 0), 1)
        if cw < w // 15 or ch < h // 2:
            continue
        
        o_img_c = cv2.rectangle(o_img_c, (cx,cy), (cx+cw,cy+ch), (0, 255, 0), 1)
        # o_img = cv2.drawContours(o_img_c, [contour], 0, (255, 255, 0), -1)
        filtered_cnts.append(contour)
        
    results[name] = len(filtered_cnts) == 1
        
    x, y, w, h = row[2]
    
    if results[name]:
        cv2.rectangle(o_img_c, (x,y), (x+w,y+h), (0, 255, 0), 2)
    else:
        cv2.rectangle(o_img_c, (x,y), (x+w,y+h), (0, 0, 255), 2)
    
# o_img = cv2.rectangle(o_img, (min_x, min_y), (max_x, max_y), 255, 2)
cv2.imwrite("./img/output.png", o_img_c)
print(results)

{'Baumann, Lysann': True, 'Beetz, Robin Georg': False, 'Beuerle, Marco': False, 'Domitrovic, Max': False, 'Druica, Mathias': True, 'Egger, Julia': True, 'Fischer, David': True, 'Fisher, Jamie': True, 'Gmeiner, Leander Gabriel Mauritius': True, 'Handschuh, Jannik ': False, 'Hogan, Finley ': True, 'Kiele, Milan': True, 'Marschall, Linus': True, 'Medwedkin, Eduard': True, 'Naas, Jasper': True, 'Nusch, Hannes': True, 'Rottweiler, Philipp': True, 'Schilling, Tobias': True, 'Schneider, AnnaSophie': False, 'Seidel, Yannick': True, 'Siegert, Daniel Valentin': True, 'Zagst, Jonas': True}


image = cv2.imread("./img/list_2.jpg")

hsv = cv2.cvtColor(image, cv2.COLOR_BGR2HSV)
lower = np.array([90, 38, 0])
upper = np.array([145, 255, 255])
mask = cv2.inRange(hsv, lower, upper)
result = cv2.bitwise_and(image, image, mask=mask)
result[mask==0] = (255, 255, 255)

cv2.imwrite("./img/output.png", result)

In [5]:
names_signatures

[('Intelligente Systeme TIK22',
  [[8, 10, 104, 403],
   [120, 8, 1247, 412],
   [1375, 4, 1103, 415],
   [1375, 4, 1103, 415]]),
 ('Baumann, Lysann',
  [[9, 422, 104, 59],
   [122, 423, 1244, 64],
   [1375, 422, 1101, 65],
   [2485, 425, 1, 58]]),
 ('Beetz, Robin Georg',
  [[10, 489, 103, 59],
   [122, 491, 1244, 63],
   [1375, 489, 1101, 66],
   [2485, 492, 1, 58]]),
 ('Beuerle, Marco',
  [[10, 557, 103, 59],
   [122, 558, 1244, 64],
   [1374, 557, 1101, 66],
   [2484, 560, 2, 58]]),
 ('Domitrovic, Max',
  [[10, 625, 104, 59],
   [123, 626, 1242, 64],
   [1374, 625, 1102, 65],
   [2485, 627, 1, 58]]),
 ('Druica, Mathias',
  [[11, 693, 103, 58],
   [123, 693, 1243, 64],
   [1375, 693, 1101, 65],
   [2485, 694, 1, 59]]),
 ('Egger, Julia',
  [[11, 760, 103, 59],
   [123, 761, 1242, 64],
   [1375, 760, 1101, 66],
   [2485, 762, 1, 39]]),
 ('Fischer, David',
  [[11, 828, 103, 58], [123, 828, 1242, 64], [1374, 828, 1103, 65]]),
 ('Fisher, Jamie',
  [[11, 895, 103, 58], [123, 896, 1243, 63]

# Row/Column detection

# Printed text detection (left column)

# Detection of valid signature fields

# Extraction of list of user and if valid signature is present